In [ ]:
# Standard libraries
from copy import deepcopy

import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "chernysheva-tmaze-analysis-2020"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from scipy.stats import mannwhitneyu, wilcoxon, binom_test, combine_pvalues

# User libraries
from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from mesostat.utils.arrays import numpy_merge_dimensions
from mesostat.stat.connectomics import offdiag_1D
from mesostat.utils.plotting import imshow, stat_annot_patches, custom_grad_cmap
from mesostat.utils.pandas_helper import get_rows_colvals

from src.lib.data_db import BehaviouralNeuronalDatabase
import src.lib.plots_lib as plots_lib
import src.lib.table_lib as table_lib
from src.lib.metric_wrapper import metric_by_selector_all, metric_by_selector
from src.lib.extra_metrics import num_non_zero_std, num_sample
import src.lib.tests_phase_signle_cell as single_cell_tests
from src.lib.significant_cells_lib import SignificantCells

%load_ext autoreload
%autoreload 2

In [ ]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
#params['root_path_data']  = gui_fpath("Path to data files", "./")
# params['root_path_dff'] = '/media/alyosha/Data/TE_data/mariadata/dff/'
# params['root_path_deconv'] = '/media/alyosha/Data/TE_data/mariadata/deconv/'
params['root_path_dff'] = '/media/aleksejs/DataHDD/work/data/maria/dff/'
params['root_path_deconv'] = '/media/aleksejs/DataHDD/work/data/maria/deconv/'

In [ ]:
dataDB = BehaviouralNeuronalDatabase(params)

In [ ]:
dataDB.read_neuro_files()

In [ ]:
dataDB.read_behavior_files()

# 1. Neuron-Time-Average

**Goal**: Attempt to predict phase from mean activity over trial

# 2 Time-Average

**Goal**: Attempt to predict phase from average activities of individual cells.
* Count predictive cells in each mouse
* Test overlap over phases/intervals - identify how many significant cells overlap

## 2.1 avg-firing-rate by neuron
 -> heatmap by mouse | 1D plot by neurons

In [ ]:
def plot_avg_firing_rate_by_neuron(datatype, phaseType, haveWaiting=True):
    settings = {"zscoreChannel" : False, "serial" : True, "metricSettings" : {}}

    for mousename in sorted(dataDB.mice):
        fig, ax = plt.subplots(ncols=4, figsize=(4*2, 6), tight_layout=True)
        fig.suptitle(mousename)
        iCol = 0

        for performance in dataDB.get_performances():
            if phaseType != 'phase':
                boundingLines = dataDB.get_phase_bounding_lines(phaseType, performance, haveWaiting=haveWaiting)

            for direction in dataDB.get_directions():
                queryDict = {'datatype' : datatype, 'mousename' : mousename, 'performance' : performance, 'direction' : direction}
                means = metric_by_selector_all(dataDB, queryDict, phaseType, 'mean', 'p', settings, haveWaiting=haveWaiting)                
                means = means.T
                
                print(means.shape)

                # Sort by max argument
                # But only based on the first one (CL) so that all 4 combinations have the same sorting
                if iCol == 0:
                    idxsMaxArgs = np.argsort(np.array([np.argmax(m) for m in means]))
                means = means[idxsMaxArgs]

                title = direction[0] + performance[0]
                imshow(fig, ax[iCol], means, limits=None, title=title, haveColorBar=True)
                if phaseType != 'phase':
                    for bline in boundingLines:
                        ax[iCol].axvline(x=bline-1, linestyle='--', color='r', alpha=0.5)

                iCol += 1

        plt.savefig(mousename + '_avgrate_' + phaseType + '.pdf', dpi=600)
        plt.close()

### 2.1.1 avg-firing-rate by neuron and interval

In [ ]:
plot_avg_firing_rate_by_neuron('zscore', 'interval', haveWaiting=False)

### 2.1.2 avg-firing-rate by neuron and phase

In [ ]:
plot_avg_firing_rate_by_neuron('zscore', 'phase', haveWaiting=False)

In [ ]:
plot_avg_firing_rate_by_neuron('zscore', 'semiphase', haveWaiting=False)

## 2.2 significant-avg-firing-rate by neuron

In [ ]:
# cmapConfusion = gen_cmap_3color(
#     np.array([255, 255, 255]),
#     np.array([168, 217, 49]),
#     np.array([255, 0, 0])
# )

cmapConfusion = custom_grad_cmap(
    np.array([
        [0,0,0],
        [255,198,126],
        [233,79,25]
    ])
)

cmapSignificance = custom_grad_cmap(
    np.array([
        [255, 255, 255],
        [233, 79, 25]
    ])
)

In [ ]:
def plot_significant_firing_rate_by_neuron(datatype, phaseType, confThr=0.01, haveWaiting=True):
    settings = {"zscoreChannel" : False, "serial" : True, "metricSettings" : {}}

    performanceValues = dataDB.get_performances()
    directionValues = dataDB.get_directions()
    
    directionValues += ['All']
    if phaseType == 'phase':
        performanceValues += ['All']
        
    nCol = len(performanceValues) * len(directionValues)
    cumulConfDict = {}
    cumulMeansSignDict = {}
    
    for mousename in sorted(dataDB.mice):
        print('doing mouse', mousename)
        nChannel = dataDB.get_nchannel(mousename, datatype)

        fig1, ax1 = plt.subplots(ncols=nCol, figsize=(nCol*2, 6), tight_layout=True)
        fig2, ax2 = plt.subplots(ncols=nCol, figsize=(nCol*4, 4), tight_layout=True)
        
        fig1.suptitle(mousename)
        fig2.suptitle(mousename)
        
        if phaseType=='phase':
            fig3, ax3 = plt.subplots(ncols=nCol, figsize=(nCol*4, 4), tight_layout=True)
            fig3.suptitle(mousename)
            
        iCol = 0

        for performance in performanceValues:
            if phaseType != 'phase':
                boundingLines = dataDB.get_phase_bounding_lines(phaseType, performance, haveWaiting=haveWaiting)
                    
            for direction in directionValues:
                queryDict = {'datatype' : datatype, 'mousename' : mousename}
                if performance != 'All':
                    queryDict = {**queryDict, **{'performance' : performance}}
                if direction != 'All':
                    queryDict = {**queryDict, **{'direction' : direction}}
                    
                # Calculate p-values
                pVals2D = single_cell_tests.test_inverse_all_selectors(dataDB, queryDict, phaseType, metricName='mean', alternative="greater", haveWaiting=haveWaiting, settings=settings)
                negLogPVals2D = -np.log10(pVals2D)
                
                # Calculate significant cells and confusion matrix
                signCellsByPhase = single_cell_tests.pvalues_2_significant_cells(pVals2D, confThr)
                confMat = single_cell_tests.significance_confusion_matrix(signCellsByPhase) / nChannel

                # Sort by max argument
                # But only based on the first one (CL) so that all 4 combinations have the same sorting
                if iCol == 0:
                    idxsMaxArgs = np.argsort(np.array([np.argmax(p) for p in negLogPVals2D]))
                negLogPVals2D = negLogPVals2D[idxsMaxArgs]

                colorBoundingLines = 'white'
                
                ###########################
                # Plot significance for each cell and phaseType
                ###########################
                cmapSignificant = 'copper' #'YlOrBr' # 'viridis'
                title = direction[0] + performance[0]
                if phaseType != 'phase':
                    imshow(fig1, ax1[iCol], negLogPVals2D, limits=[0, 4], title=title, haveColorBar=True, cmap=cmapSignificant)
                    for bline in boundingLines:
                        ax1[iCol].axvline(x=bline-1, linestyle='--', color=colorBoundingLines, alpha=1)
                else:
                    imshow(fig1, ax1[iCol], negLogPVals2D, limits=[0, 4], title=title, haveTicks=True, haveColorBar=False, cmap=cmapSignificant)
                    
                ax1[iCol].set_yticks(np.arange(0, len(negLogPVals2D), 10))

                ###########################
                # Plot confusion matrix
                ###########################
                imshow(fig2, ax2[iCol], confMat, limits=[0, 1], title=title, haveColorBar=True, cmap=cmapConfusion)
                if phaseType != 'phase':
                    for bline in boundingLines:
                        ax2[iCol].axvline(x=bline-1, linestyle='--', color=colorBoundingLines, alpha=1)
                        ax2[iCol].axhline(y=bline-1, linestyle='--', color=colorBoundingLines, alpha=1)
                
                # Store confusion matrices 
                cumulConfDict[(mousename, direction, performance)] = confMat
                
                #####################################
                # Compare rate/DFF for cells of different phases
                #####################################
                if phaseType == 'phase':
                    meansByPhase = metric_by_selector_all(dataDB, queryDict, phaseType, 'mean', 'p', settings, haveWaiting=haveWaiting)
                    meansSignByPhase = [means[list(idxs)] for means, idxs in zip(meansByPhase, signCellsByPhase)]

                    phaseNames = dataDB.get_phasetype_keys(phaseType, 'Correct', haveWaiting=haveWaiting)
                    
                    sigTestPairs = []
                    nSignCells = [len(s) for s in signCellsByPhase]
                    if (nSignCells[0] > 1) and (nSignCells[1] > 1):
                        sigTestPairs += [(0,1)]
                    if (nSignCells[1] > 1) and (nSignCells[2] > 1):
                        sigTestPairs += [(1,2)]
                    if len(sigTestPairs) == 0:
                        sigTestPairs = None
                    
                    ax3[iCol].set_title(title)
                    plots_lib.plot_labeled_violins(ax3[iCol], meansSignByPhase, phaseNames, phaseType, "mean",
                                                   joinMeans=True, printLogP=False, sigTestPairs=sigTestPairs,
                                                   violinInner='point')
                    
                    # Combine means for all mice
                    cumulMeansSignDict[(mousename, direction, performance)] = meansSignByPhase
                
                iCol += 1

        fig1.savefig(mousename + '_significantrate_' + datatype + '_' + phaseType + '.pdf', dpi=600)
        plt.close()
        fig2.savefig(mousename + '_significantrate_' + datatype + '_' + phaseType + '_confusion.pdf', dpi=600)
        plt.close()
        
        if phaseType == 'phase':
            fig3.savefig(mousename + '_significantcell_' + datatype + '_' + phaseType + '_rate_comparison.pdf', dpi=600)
            plt.close()

    iCol = 0
    fig2All, ax2All = plt.subplots(ncols=nCol, figsize=(nCol*4, 4), tight_layout=True)    
    fig3All, ax3All = plt.subplots(ncols=nCol, figsize=(nCol*4, 4), tight_layout=True)    
    for performance in performanceValues:
        for direction in directionValues:            
            title = direction[0] + performance[0]
            
            ###########################
            # Plot confusion matrix
            ###########################
            confThisCondition = [cumulConfDict[(mousename, direction, performance)] for mousename in dataDB.mice]
            confAvg = np.mean(confThisCondition, axis=0)
            
            imshow(fig2All, ax2All[iCol], confAvg, limits=[0, 1], title=title, haveColorBar=True, cmap=cmapConfusion)
            if phaseType != 'phase':
                boundingLines = dataDB.get_phase_bounding_lines(phaseType, performance, haveWaiting=haveWaiting)
                
                for bline in boundingLines:
                    axAll[iCol].axvline(x=bline-1, linestyle='--', color=colorBoundingLines, alpha=1)
                    axAll[iCol].axhline(y=bline-1, linestyle='--', color=colorBoundingLines, alpha=1)
                    
            #####################################
            # Compare rate/DFF for cells of different phases
            #####################################
            if phaseType == 'phase':
                phaseNames = dataDB.get_phasetype_keys(phaseType, 'Correct', haveWaiting=haveWaiting)
                
                meansThisParam = [cumulMeansSignDict[(mousename, direction, performance)] for mousename in dataDB.mice]
                meansThisParam = [np.hstack([m[i] for m in meansThisParam]) for i in range(len(phaseNames))]
                
                ax3All[iCol].set_title(title)
                plots_lib.plot_labeled_violins(ax3All[iCol], meansThisParam, phaseNames, phaseType, "mean",
                                               joinMeans=True, printLogP=False, sigTestPairs=sigTestPairs,
                                               violinInner='point')
            
            iCol += 1
            
    fig2All.savefig('Allmice_significantrate_' + datatype + '_' + phaseType + '_confusion.pdf', dpi=600)
    plt.close()
    
    if phaseType == 'phase':
        fig3All.savefig('Allmice_significantcell_' + datatype + '_' + phaseType + '_rate_comparison.pdf', dpi=600)
        plt.close()

In [ ]:
for datatype in ['deconv']:#['deconv', 'raw']:
    for phasetype in ['phase']:#['semiphase', 'phase', 'interval']:
        print(datatype, phasetype)
        plot_significant_firing_rate_by_neuron(datatype, phasetype, confThr=0.01, haveWaiting=False)

## 2.3 Storing significant neurons, counting across mice

In [ ]:
def plot_df_count_by_mice(ax, data1, data2, label1, label2, xLabels, nCellPerMouse):
    dataNorm1 = np.array(data1) / np.array(nCellPerMouse)
    dataNorm2 = np.array(data2) / np.array(nCellPerMouse)
    pVals = [binom_test(d1, d1 + d2) for d1, d2 in zip(data1, data2)]
    
    xInd = np.arange(len(xLabels))
    width = 0.35  # the width of the bars
    
    rects1 = ax.bar(xInd - width/2, dataNorm1, width, label=label1)
    rects2 = ax.bar(xInd + width/2, dataNorm2, width, label=label2)
    
    for patch1, patch2, pVal in zip(rects1.patches, rects2.patches, pVals):
        stat_annot_patches(ax, patch1, patch2, pVal, fontsize=20)
    
    ax.set_xticks(xInd)
    ax.set_xticklabels(xLabels)
    
def plot_df_count_combined(ax, data1, data2, label1, label2, nCellPerMouse):
    # Convert from counts to fractions
    dataNorm1 = np.array(data1) / np.array(nCellPerMouse)
    dataNorm2 = np.array(data2) / np.array(nCellPerMouse)
    dataMean1 = np.mean(dataNorm1)
    dataMean2 = np.mean(dataNorm2)
    
    # Compute p-values and combine
    pVals = [binom_test(d1, d1 + d2) for d1, d2 in zip(data1, data2)]
    pValMean = combine_pvalues(pVals)[1]
    
    # Plot Bars
    width = 0.30  # the width of the bars
    dist = 0.05
    rects1 = ax.bar([-(width + dist)/2], [dataMean1], width, label=label1)
    rects2 = ax.bar([+(width + dist)/2], [dataMean2], width, label=label2)
        
    # Plot individual lines
    for d1, d2 in zip(dataNorm1, dataNorm2):
        ax.plot([-(width + dist)/2, (width + dist)/2], [d1, d2], color='gray')
        
    # Annotate bars
    for patch1, patch2 in zip(rects1.patches, rects2.patches):
        stat_annot_patches(ax, patch1, patch2, pValMean, fontsize=20)
        
    ax.set_xticks([])
    ax.set_ylim([0, 1.1*np.max(np.hstack([dataNorm1, dataNorm2]))])


def plot_save_significantly_firing_neurons(datatype, phaseType, exclusiveIndices, exclusiveLabels, ranges=None, confThr=0.01, haveAll=False):
    settings = {"zscoreChannel" : False, "serial" : True, "metricSettings" : {}}
    
    exclusiveLabelsKey = '_'.join(exclusiveLabels)

    significantCellsDict = {label : [] for label in exclusiveLabels}
    
    if not haveAll:
        performanceValues = ['Correct', 'Mistake']
        directionValues = ['L', 'R']
    else:
        performanceValues = ['Correct', 'Mistake', 'All']
        directionValues = ['L', 'R', 'All']
    
    mice = list(sorted(dataDB.mice))
    for mousename in mice:
        print('doing mouse', mousename)

        for performance in performanceValues:
            for direction in directionValues:
                queryDict = {'datatype' : datatype, 'mousename' : mousename}
                if performance != 'All':
                    queryDict = {**queryDict, **{'performance' : performance}}
                if direction != 'All':
                    queryDict = {**queryDict, **{'direction' : direction}}
                    
                # Calculate p-values and significant cells
                pVals2D = single_cell_tests.test_inverse_all_selectors(dataDB, queryDict, phaseType, metricName='mean', alternative="greater", settings=settings, ranges=ranges)
                signCellsByPhase = single_cell_tests.pvalues_2_significant_cells(pVals2D, confThr)

                # Determine maintenance-significant cells for storage
                exclusiveSets = single_cell_tests.find_exclusive_sets(signCellsByPhase, exclusiveIndices)

                for thisSet, thisLabel in zip(exclusiveSets, exclusiveLabels):
                    significantCellsDict[thisLabel] += [[mousename, performance, direction, np.array(list(thisSet))]]

    # Store significant cells in HDF5
    significantCellsDFDict = {}
    for label, signCellsData in significantCellsDict.items():
        dfCells = pd.DataFrame(signCellsData, columns=['mousename', 'performance', 'direction', 'cells'])
        dfCells.to_hdf('significant_cells_' + datatype + '_' + label + '.h5', key='df', mode='w')
        significantCellsDFDict[label] = dfCells
        
    nCellPerMouse = np.array([dataDB.get_nchannel(mousename, datatype) for mousename in mice])

    # Plot selector-specific number of significant cells vs mice
    for performance in performanceValues:
        for direction in directionValues:
            queryDict = {"performance" : performance, "direction" : direction}
            
            nSignCellByMouse = []
            for label, signCellsDF in significantCellsDFDict.items():
                rows = get_rows_colvals(signCellsDF, queryDict, dropQuery=True)
                nSignCellByMouse += [[len(list(row.values)[1]) for idx, row in rows.iterrows()]]
            
            ######################
            #  Plot by mouse
            ######################
            
            figCount, axCount = plt.subplots(figsize=(5, 5))
            plot_df_count_by_mice(axCount, *nSignCellByMouse, *exclusiveLabels, mice, nCellPerMouse)
                
            figKey = '_'.join([exclusiveLabelsKey, datatype, phaseType, performance, direction])
                
            axCount.legend()
            figCount.savefig('significantrate_by_mouse_' + figKey + '.pdf')
            plt.close()
            
            ######################
            #  Plot combined
            ######################
            
            figCountAll, axCountAll = plt.subplots(figsize=(5, 5))
            plot_df_count_combined(axCountAll, *nSignCellByMouse, *exclusiveLabels, nCellPerMouse)
            
            figCountAll.savefig('significantrate_combined_' + figKey + '.pdf')
            plt.close()

In [ ]:
# Testing each phase vs remainder of the trial, storing maintenance vs encoding+retrieval
# for datatype in ['raw', 'deconv']:
#     plot_save_significantly_firing_neurons(datatype,
#                                            'phase',
#                                            [[1], [0, 2]],
#                                            ['mt', 'enc_ret'],
#                                            confThr=0.01,
#                                            haveAll=True)
    
    
for datatype in ['raw', 'deconv']:
    plot_save_significantly_firing_neurons(datatype,
                                           'phase',
                                           [[0], [2]],
                                           ['end', 'ret'],
                                           confThr=0.01,
                                           haveAll=True)

In [ ]:
for datatype in ['raw', 'deconv']:
    plot_save_significantly_firing_neurons(datatype,
                                           'interval', 
                                           [[0], [1]],
                                           ['enc_base', 'enc_reward'],
                                           confThr=0.01,
                                           ranges=[2, 4],
                                           haveAll=True)

## 2.4 Load and test significant cells

In [ ]:
significantCellsSelectorDatatype = {}

for datatype in ['raw', 'deconv']:
    signCellsMaintenance = SignificantCells('significant_cells_'+datatype+'_mt.h5').get_cells_by_mouse()
    signCellsReward = SignificantCells('significant_cells_'+datatype+'_enc_reward.h5').get_cells_by_mouse()

    significantCellsSelectorDatatype[datatype] = {
        'None' : None,
        'Maintenance' : signCellsMaintenance,
        'Reward' : signCellsReward
    }

## 2.5 Are 'Maintenance Cells' more active than 'Encoding Cells'?

1. For each cell, compute phase-avg over time
2. For each cell, for each trial, compute ratio of ENC vs MT
3. 

$$R_{i,j,ENC} = \int_{ENC} R_{i,j}(t)$$
$$R_{i,j,MT} = \int_{MT} R_{i,j}(t)$$
$$\phi_{i,j} = \frac{R_{i,j,ENC}}{R_{i,j,MT}}$$
$$\bar \phi_{i} = \frac{1}{N_{trial}} \sum_j \phi_{i,j}$$


In [ ]:
def plot_ratio_enc_mt(datatype):
    settings = {"zscoreChannel" : False, "serial" : True, "metricSettings" : {}}

    nPlots = len(dataDB.mice)
    fig, ax = plt.subplots(ncols=nPlots, figsize=(4*nPlots, 4), tight_layout=True)

    for iMouse, mousename in enumerate(sorted(dataDB.mice)):
        queryDict = {'datatype' : datatype, 'mousename' : mousename}

        rezEnc = metric_by_selector(dataDB, queryDict, 'mean', 'p', {'phase' : 'Encoding'}, settings, channelFilter=None)
        rezMt = metric_by_selector(dataDB, queryDict, 'mean', 'p', {'phase' : 'Maintenance'}, settings, channelFilter=None)
            
        rezRatio = (rezMt - rezEnc) / (rezMt + rezEnc)
        
        cdf = lambda x: (np.sort(x), np.linspace(0, 1, len(x)))
        
        cdfX, cdfY = cdf(rezRatio)
        idx0 = np.argmin(cdfX**2)
        yMin = cdfY[idx0]
        
        ax[iMouse].plot(cdfX, cdfY)
        ax[iMouse].set_title(mousename)
        ax[iMouse].axvline(x=0, linestyle='--', color='r')
        ax[iMouse].axhline(y=yMin, linestyle='--', color='r')
        
    plt.show()

plot_ratio_enc_mt('deconv')

# 3.1 Table - Discriminate Phases by Metric

In [ ]:
%%time
dataDB.verbose = False

phases = ['Encoding', 'Maintenance', 'Retrieval']
#settings = {"serial" : True, "metricSettings" : {"metric" : num_non_zero_std}}
settings = {"serial" : True, "metricSettings" :{"max_lag" : 1}}
sweepDict = {
    #"mousename" : sorted(list(dataDB.mice)),
    "datatype": ["deconv"],#, "raw", "high", "deconv"],
    "performance": ["Correct", "Mistake", "All"],
    "direction": ["L", "R", "All"]
}

table_lib.table_discriminate_time(dataDB, sweepDict,
                                         {"phase" : phases},
                                         "mean",
                                         trgDimOrder="r",
                                         settings=settings, multiplexKey="mousename")

# 3.2 Violins - Metric by Phase and Interval

In [ ]:
def plot_violins(datatype, phaseType, metricName, settings, haveWaiting=True, signCellsSelector=None):
    if signCellsSelector == None:
        signCellsSelector = {'None' : None}
        
    signCellsKey, signCellsMouseDict = list(signCellsSelector.items())[0]
    
    
    for performance in ["Correct", "Mistake"]:
        fig, ax = plt.subplots(ncols=3, figsize=(15,5))
        fig.suptitle(performance)

        intervalOrigIndices = dataDB.get_phasetype_keys(phaseType, performance)
        boundingLines = dataDB.get_phase_bounding_lines(phaseType, performance, haveWaiting=haveWaiting) - 1
        phases = dataDB.get_phasetype_keys('phase', performance, haveWaiting=haveWaiting)

        for mousename in sorted(dataDB.mice)[:-1]:
            queryThis = {"datatype" : datatype, "mousename" : mousename, "performance" : performance}  # "direction" : "R"
            rez2D = metric_by_selector_all(dataDB, queryThis, phaseType, metricName, 'p', settings, haveWaiting=haveWaiting, channelFilter=signCellsMouseDict)
    #         rez2D = rez2D[..., 0]
            plots_lib.plot_labeled_bars(ax[0], rez2D, intervalOrigIndices, plotLabel=mousename, alpha=0.2, vlines=boundingLines)

        rez2DIntervAll = metric_by_selector_all(dataDB, queryThis, phaseType, metricName, 'p', settings, haveWaiting=haveWaiting, channelFilter=signCellsMouseDict)
        rez2DPhaseAll = metric_by_selector_all(dataDB, queryThis, 'phase', metricName, 'p', settings, haveWaiting=haveWaiting, channelFilter=signCellsMouseDict)
        plots_lib.plot_labeled_bars(ax[1], rez2DIntervAll, intervalOrigIndices, vlines=boundingLines)
        plots_lib.plot_labeled_violins(ax[2], rez2DPhaseAll, phases, "phase", metricName, joinMeans=True, printLogP=True)

        ax[0].legend()
        plt.savefig(phaseType+'_'+datatype+'_'+signCellsKey+'_avg_violins_'+performance+'.pdf')
        plt.show()

In [ ]:
settings = {"serial" : True, "metricSettings" : {"max_lag" : 1}}
#settings = {"serial" : True, "metricSettings" : {"metric" : num_non_zero_std}}

        
for datatype in ['raw', 'deconv']:
    for phaseType in ['interval', 'semiphase']:
        for signCellsName, signCells in significantCellsSelectorDatatype[datatype].items():
            print("datatype", datatype, "Significant Cells :", signCellsName)
            plot_violins(datatype, phaseType, "mean", settings, haveWaiting=False, signCellsSelector={signCellsName:signCells})